In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install soundfile

In [ ]:
import os
import pandas as pd
import librosa
import requests
import soundfile as sf # Import soundfile
from datasets import load_dataset, Dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor, Trainer, TrainingArguments
from evaluate import load as load_metric # Import load_metric from evaluate

# Download and organize paths from metadata
# Assume: meta.csv contains columns audio_path, transcription_path

df = pd.read_csv("/content/drive/MyDrive/data/Voice_rec/FT Data - data.csv")

# Create directories to store processed files
os.makedirs("/content/drive/MyDrive/processed1", exist_ok=True)
os.makedirs("/content/drive/MyDrive/transcriptions1", exist_ok=True) # Create a directory for transcriptions

processed_data = [] # Initialize the list to store processed data

for i, row in df.iterrows():
    audio_url = row['rec_url_gcp']
    transcription_url = row['transcription_url_gcp'] # Get transcription URL

    # Construct the local filenames
    audio_filename = os.path.join("/content/drive/MyDrive/processed1", os.path.basename(audio_url))
    transcription_filename = os.path.join("/content/drive/MyDrive/transcriptions1", os.path.basename(transcription_url))


    # Download and process audio file
    print(f"Attempting to download audio: {audio_url}")
    try:
        response = requests.get(audio_url, stream=True)
        response.raise_for_status()
        with open(audio_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Successfully downloaded audio: {audio_filename}")

        # Verify the audio file exists before trying to load and process
        if os.path.exists(audio_filename):
            print(f"Audio file exists locally: {audio_filename}")
            # Convert to standard sample rate
            y, sr = librosa.load(audio_filename, sr=16000)
            sf.write(audio_filename, y, sr)
            print(f"Successfully processed and saved audio: {audio_filename}")

            # Download transcription
            print(f"Attempting to download transcription: {transcription_url}")
            response = requests.get(transcription_url)
            response.raise_for_status()
            transcription_text = response.text

            # Save the transcription to a local file (optional, but good practice)
            with open(transcription_filename, 'w') as f:
                f.write(transcription_text)
            print(f"Successfully downloaded transcription: {transcription_filename}")

            # Append processed data to the list
            processed_data.append({"audio_path": audio_filename, "transcription": transcription_text})

        else:
            print(f"Downloaded audio file not found locally: {audio_filename}")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
    except Exception as e:
        print(f"Error processing file: {e}")

# Create a new DataFrame from processed data after the loop
processed_df = pd.DataFrame(processed_data)
save_path = "/content/drive/MyDrive/data/Voice_rec/processed_data.csv"
processed_df.to_csv(save_path, index=False)
print(f"Processed data saved to: {save_path}")

# Display the first few rows of processed_df to verify
display(processed_df.head())

Attempting to download audio: https://storage.googleapis.com/joshtalks-data-collection/hq_data/hi/967179/825780_audio.wav
Successfully downloaded audio: /content/drive/MyDrive/processed1/825780_audio.wav
Audio file exists locally: /content/drive/MyDrive/processed1/825780_audio.wav
Successfully processed and saved audio: /content/drive/MyDrive/processed1/825780_audio.wav
Attempting to download transcription: https://storage.googleapis.com/joshtalks-data-collection/hq_data/hi/967179/825780_transcription.json
Successfully downloaded transcription: /content/drive/MyDrive/transcriptions1/825780_transcription.json
Attempting to download audio: https://storage.googleapis.com/joshtalks-data-collection/hq_data/hi/967179/825727_audio.wav
Successfully downloaded audio: /content/drive/MyDrive/processed1/825727_audio.wav
Audio file exists locally: /content/drive/MyDrive/processed1/825727_audio.wav
Successfully processed and saved audio: /content/drive/MyDrive/processed1/825727_audio.wav
Attempting 

,audio_path,transcription
0,/content/drive/MyDrive/processed1/825780_audio...,"[\n {\n ""start"": 0.11,\n ""end"": 14.42,\..."
1,/content/drive/MyDrive/processed1/825727_audio...,"[\n {\n ""start"": 0.47,\n ""end"": 0.8,\n ..."
2,/content/drive/MyDrive/processed1/988596_audio...,"[\n {\n ""start"": 0.53,\n ""end"": 12.11,\..."
3,/content/drive/MyDrive/processed1/990175_audio...,"[\n {\n ""start"": 5.51,\n ""end"": 11.6,\n..."
4,/content/drive/MyDrive/processed1/526266_audio...,"[\n {\n ""start"": 1.69,\n ""end"": 15.7,\n..."


In [ ]:
import pandas as pd
processed_df = pd.read_csv("/content/drive/MyDrive/data/Voice_rec/processed_data.csv")

In [ ]:
processed_df.head()

,audio_path,transcription
0,/content/drive/MyDrive/processed1/825780_audio...,"[\n {\n ""start"": 0.11,\n ""end"": 14.42,\..."
1,/content/drive/MyDrive/processed1/825727_audio...,"[\n {\n ""start"": 0.47,\n ""end"": 0.8,\n ..."
2,/content/drive/MyDrive/processed1/988596_audio...,"[\n {\n ""start"": 0.53,\n ""end"": 12.11,\..."
3,/content/drive/MyDrive/processed1/990175_audio...,"[\n {\n ""start"": 5.51,\n ""end"": 11.6,\n..."
4,/content/drive/MyDrive/processed1/526266_audio...,"[\n {\n ""start"": 1.69,\n ""end"": 15.7,\n..."


In [ ]:
import os
import pandas as pd
import librosa
import requests
import soundfile as sf # Import soundfile
from datasets import load_dataset, Dataset
import torchaudio
from pydub import AudioSegment
from transformers import WhisperForConditionalGeneration, WhisperProcessor, Trainer, TrainingArguments
from evaluate import load as load_metric # Import load_metric from evaluate

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [ ]:
def segment_audio(audio_path, transcript, out_folder, max_sec=30):
    audio = AudioSegment.from_wav(audio_path)
    duration = audio.duration_seconds
    # with open(transcript_path, 'r', encoding='utf-8') as tf:
    #     transcript = tf.read().strip()
    base = os.path.splitext(os.path.basename(audio_path))[0]
    n_splits = int(duration // max_sec) + 1
    segments = []
    chars_per_split = len(transcript) // n_splits

    for i in range(n_splits):
        start_ms = i * max_sec * 1000
        end_ms = min((i+1) * max_sec * 1000, len(audio))
        seg = audio[start_ms:end_ms]
        seg_file = f"{out_folder}/{base}_seg{i}.wav"
        seg.export(seg_file, format="wav")

        # basic split: proportional assignment
        seg_transcript = transcript[i*chars_per_split : (i+1)*chars_per_split] if i < n_splits - 1 else transcript[i*chars_per_split :]
        segments.append({'audio': seg_file, 'transcript': seg_transcript})
    return segments

all_segments = []
# Create the 'split_audio' directory if it doesn't exist
os.makedirs('split_audio', exist_ok=True)
for idx, row in processed_df.iterrows():
    audio_path = row['audio_path']
    transcript = row['transcription']
    all_segments.extend(segment_audio(audio_path, transcript, 'split_audio'))

# Create manifest for training
manifest = pd.DataFrame(all_segments)
manifest.to_csv('whisper_train_manifest.csv', index=False)

In [ ]:
import pandas as pd
pd.read_csv("/content/drive/MyDrive/whisper_train_manifest.csv")

,audio,transcript
0,split_audio/825780_audio_seg0.wav,"[\n {\n ""start"": 0.11,\n ""end"": 14.42,\..."
1,split_audio/825780_audio_seg1.wav,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...
2,split_audio/825780_audio_seg2.wav,"नीचे""\n },\n {\n ""start"": 42.47,\n ""en..."
3,split_audio/825780_audio_seg3.wav,तो जब पता जैसी रात हुआ ना शाम मतलब छै सात में ...
4,split_audio/825780_audio_seg4.wav,ता सब अपना अपना कैम्प डाल के रह रहा था और हम अ...
...,...,...
2495,split_audio/840781_audio_seg32.wav,ें कही पे भी नहीं है तो अगर हम दो हजार पैतालीस...
2496,split_audio/840781_audio_seg33.wav,""": ""जी जी""\n },\n {\n ""start"": 993.07,\n ..."
2497,split_audio/840781_audio_seg34.wav,से बड़ा तो हाथ उनका जो आपके इलाके का डीएम होता...
2498,split_audio/840781_audio_seg35.wav,हजार पैंतालीस तक अगर इसमें बदलाव नहीं आएगा लोग...


In [ ]:
# Imports
# -------------------------------
import pandas as pd
import torch
import torchaudio
import numpy as np
from datasets import Dataset, load_from_disk
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, TrainingArguments
from jiwer import wer

# -------------------------------
# Constants
# -------------------------------
MODEL_NAME = "openai/whisper-small"
MAX_LABEL_LENGTH = 448
TARGET_MEL_LENGTH = 3000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------------
#  Load processor & model
# -------------------------------
processor = WhisperProcessor.from_pretrained(MODEL_NAME)
feature_extractor = processor.feature_extractor
tokenizer = processor.tokenizer
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)

# -------------------------------
#  Preprocess function
# -------------------------------
def pad_or_truncate_mel(features, target_len=TARGET_MEL_LENGTH):
    n_mels, time_steps = features.shape
    if time_steps < target_len:
        features = np.pad(features, ((0,0),(0,target_len-time_steps)), mode="constant")
    else:
        features = features[:, :target_len]
    return features

def preprocess(example, feature_extractor, tokenizer):
    # Load audio
    audio_input, _ = torchaudio.load(example['audio'])
    audio_input = audio_input.squeeze().numpy()

    # Extract mel features
    features = feature_extractor(audio_input, sampling_rate=16000)
    mel = features.input_features[0]  # shape [80, time_steps]
    mel = pad_or_truncate_mel(mel)

    input_features = torch.tensor(mel).transpose(0,1)  # shape [3000, 80]

    # Tokenize transcript and truncate
    labels = tokenizer(
        example['transcript'],
        return_tensors="pt",
        max_length=MAX_LABEL_LENGTH,
        truncation=True
    ).input_ids.squeeze()

    return {"input_features": input_features, "labels": labels}

# -------------------------------
#  Load your CSV manifest
# -------------------------------
df = pd.read_csv("/content/drive/MyDrive/whisper_train_manifest_fullpath.csv")


dataset = Dataset.from_pandas(df)

# -------------------------------
# Preprocess dataset
# -------------------------------
dataset = dataset.map(
    lambda x: preprocess(x, feature_extractor=feature_extractor, tokenizer=tokenizer),
    remove_columns=dataset.column_names
)

# Save for future use
dataset.save_to_disk("/content/drive/MyDrive/processed_train_dataset")



Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Saving the dataset (0/5 shards):   0%|          | 0/2500 [00:00<?, ? examples/s]

In [ ]:
pd.read_csv("/content/drive/MyDrive/whisper_train_manifest_fullpath.csv")


,audio,transcript
0,/content/drive/MyDrive/whisper_audio/split_aud...,"[\n {\n ""start"": 0.11,\n ""end"": 14.42,\..."
1,/content/drive/MyDrive/whisper_audio/split_aud...,अनुभव करके कुछ लिखना था तो वह तो बिना देखिए नह...
2,/content/drive/MyDrive/whisper_audio/split_aud...,"नीचे""\n },\n {\n ""start"": 42.47,\n ""en..."
3,/content/drive/MyDrive/whisper_audio/split_aud...,तो जब पता जैसी रात हुआ ना शाम मतलब छै सात में ...
4,/content/drive/MyDrive/whisper_audio/split_aud...,ता सब अपना अपना कैम्प डाल के रह रहा था और हम अ...
...,...,...
2495,/content/drive/MyDrive/whisper_audio/split_aud...,ें कही पे भी नहीं है तो अगर हम दो हजार पैतालीस...
2496,/content/drive/MyDrive/whisper_audio/split_aud...,""": ""जी जी""\n },\n {\n ""start"": 993.07,\n ..."
2497,/content/drive/MyDrive/whisper_audio/split_aud...,से बड़ा तो हाथ उनका जो आपके इलाके का डीएम होता...
2498,/content/drive/MyDrive/whisper_audio/split_aud...,हजार पैंतालीस तक अगर इसमें बदलाव नहीं आएगा लोग...


In [ ]:
# --- Imports ---
import torch
from torch.utils.data import DataLoader
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import load_from_disk
import torchaudio
import pandas as pd
from jiwer import wer
from torch.nn.utils.rnn import pad_sequence

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# --- Load processor and model ---
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

# --- Load preprocessed dataset ---
train_dataset = load_from_disk("/content/drive/MyDrive/processed_train_dataset")

# --- Corrected collate function ---
def collate_fn(batch):
    # Convert input_features to tensors, remove channel dim, transpose to [seq_len, feature_dim]
    input_features = [torch.tensor(x['input_features']).squeeze(0).transpose(0,1) for x in batch]
    input_features_padded = pad_sequence(input_features, batch_first=True, padding_value=0.0)

    # Pad labels
    labels = [torch.tensor(x['labels']) for x in batch]
    labels_padded = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        "input_features": input_features_padded,
        "labels": labels_padded
    }

# --- DataLoader ---
train_loader = DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collate_fn
)

# --- Optimizer ---
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# --- Training Loop ---
model.train()
for epoch in range(3):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss / len(train_loader):.4f}")

# --- Save model & processor ---
model.save_pretrained("/content/drive/MyDrive/finetuned_model")
processor.save_pretrained("/content/drive/MyDrive/finetuned_model")




Using device: cuda


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch 1 Loss: 0.9648
Epoch 2 Loss: 0.7514
Epoch 3 Loss: 0.6722


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3922: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[]

In [ ]:
from datasets import load_dataset

# Load only the test split
test_ds = load_dataset("Ritwika03/hindi_google_fleurs", split="test")

print(test_ds)



README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00004.parquet:   0%|          | 0.00/462M [00:00<?, ?B/s]

data/train-00001-of-00004.parquet:   0%|          | 0.00/465M [00:00<?, ?B/s]

data/train-00002-of-00004.parquet:   0%|          | 0.00/452M [00:00<?, ?B/s]

data/train-00003-of-00004.parquet:   0%|          | 0.00/468M [00:00<?, ?B/s]

...

data/eval-00000-of-00001.parquet:   0%|          | 0.00/41.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2575 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/138 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/56 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'file_name', 'text', 'utterance_pitch_mean', 'utterance_pitch_std', 'snr', 'c50', 'speaking_rate', 'phonemes', 'stoi', 'si-sdr', 'pesq', 'noise', 'reverberation', 'speech_monotony', 'sdr_noise', 'pesq_speech_quality', 'text_description'],
    num_rows: 138
})


In [ ]:
test_ds

Dataset({
    features: ['audio', 'file_name', 'text', 'utterance_pitch_mean', 'utterance_pitch_std', 'snr', 'c50', 'speaking_rate', 'phonemes', 'stoi', 'si-sdr', 'pesq', 'noise', 'reverberation', 'speech_monotony', 'sdr_noise', 'pesq_speech_quality', 'text_description'],
    num_rows: 138
})

In [ ]:
# Use only test split (small, clean, and balanced)
subset = test_ds.remove_columns([
    col for col in test_ds.column_names
    if col not in ['audio', 'text']
])
subset.to_parquet("/content/drive/MyDrive/fleurs_hindi_test_audio_text.parquet")


Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

104102470

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# Use your fine-tuned model path or Hugging Face model
model_path = "/content/drive/MyDrive/finetuned_model"

processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)


In [ ]:
from datasets import Dataset

test_ds = Dataset.from_parquet("/content/drive/MyDrive/fleurs_hindi_test_audio_text.parquet")
print(test_ds)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['audio', 'text'],
    num_rows: 138
})


In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.9 MB/s  0:00:00


In [ ]:
import torch
from datasets import load_dataset
import evaluate
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import pandas as pd

In [ ]:
# STEP 3 — Load Fine-tuned Model and Processor
# ========================================
model_path = "/content/drive/MyDrive/finetuned_model"  # change if needed

processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model = model.to("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# Local Parquet file
ds = load_dataset("parquet", data_files="/content/drive/MyDrive/fleurs_hindi_test_audio_text.parquet")

In [ ]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.5 MB/s eta 0:00:00


In [ ]:
# ========================================
# STEP 1 — Install Dependencies
# ========================================
!pip install transformers datasets evaluate torchaudio soundfile -q

import torch
from datasets import load_dataset
import evaluate
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import pandas as pd
import torchaudio
from google.colab import drive

# ========================================
#Mount Google Drive
# ========================================


# ========================================
# Load Fine-tuned Whisper Model
# ========================================
model_path = "/content/drive/MyDrive/finetuned_model"  # <-- change if needed

processor = WhisperProcessor.from_pretrained(model_path)
model = WhisperForConditionalGeneration.from_pretrained(model_path)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

# ========================================
# Load FLEURS Hindi Test Dataset
# ========================================
# Load saved parquet from Drive
ds = load_dataset("parquet", data_files="/content/drive/MyDrive/fleurs_hindi_test_audio_text.parquet")
print(ds)

# ========================================
# Preprocess + Resample Audio
# ========================================
def preprocess_function(batch):
    waveform = batch["audio"]["array"]
    sr = batch["audio"]["sampling_rate"]

    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(torch.tensor(waveform).float())

    waveform = waveform / waveform.abs().max()  # normalize

    inputs = processor.feature_extractor(
        waveform.squeeze().numpy(), sampling_rate=16000, return_tensors="pt"
    )
    batch["input_features"] = inputs.input_features[0]
    batch["reference_text"] = batch["text"]
    return batch


ds = ds.map(preprocess_function)
test_ds = ds["train"]  # select the split


# ========================================
#  Batch Inference on GPU
# ========================================
predictions, references = [], []
batch_size = 4

for i in range(0, len(test_ds), batch_size):
    batch = test_ds.select(range(i, min(i+batch_size, len(test_ds))))
    input_features = torch.stack([torch.tensor(x["input_features"]) for x in batch]).to(device)

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            num_beams=5,
            early_stopping=True
        )

    decoded = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    predictions.extend(decoded)
    references.extend([x["text"] for x in batch])


# ========================================
#  Compute WER
# ========================================
wer_metric = evaluate.load("wer")
wer_score = wer_metric.compute(predictions=predictions, references=references)
print(f"\n Word Error Rate (WER): {wer_score:.3f}")

# ========================================
#  Save Predictions to Drive
# ========================================
results_df = pd.DataFrame({
    "reference": references,
    "prediction": predictions
})
results_df.to_csv("/content/drive/MyDrive/whisper_fleurs_hindi_eval_results.csv", index=False)
print("\n Results saved to: /content/drive/MyDrive/whisper_fleurs_hindi_eval_results.csv")

# ========================================
# Inspect Sample Predictions
# ========================================
for j in range(5):
    print(f"\nReference: {references[j]}")
    print(f"Prediction: {predictions[j]}")


DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 138
    })
})


Map:   0%|          | 0/138 [00:00<?, ? examples/s]


✅ Word Error Rate (WER): 0.874

📁 Results saved to: /content/drive/MyDrive/whisper_fleurs_hindi_eval_results.csv

Reference: जंगल में जीवित रहने वाली पुस्तकें और पत्रिकाएं आम हैं लेकिन युद्ध क्षेत्रों से निपटने वाले प्रकाशन बहुत कम हैं
Prediction:  चंगल में जीवित रहने वाले खुश्तकर पत्रिका है याम है लेकिन डूध छेत्रों से रिपटने वाले प्रकासम बहुत कब है चंगल में जीवित रहने वाले खुश्तकर पत्रिका है याम है लेकिन डूध छेत्रों से रिपटने वाले प्रकासम बहुत कब है चंगल में जीवित रहने वाले खुश्तकर 

Reference: दक्षिणी ध्रुव पारगमन या राजमार्ग रॉस सागर पर मैक्मुर्डो स्टेशन से ध्रुव तक 1600 किमी का रास्ता है
Prediction:  मैक मोड्डो स्टेशन से द्रूव तक सोला सो किलोमीटर का रास्ता है ।
 ।
 ।
 ।
 ।
 ।
   

Reference: उचित पोषण संबंधी सेवन अकेले कुलीन प्रदर्शन उत्पन्न नहीं कर सकते लेकिन वे युवा एथलीटों के संपूर्ण स्वास्थ्य को महत्वपूर्ण रूप से प्रभावित कर सकते हैं
Prediction:  कुलीन पदर्शन उत्पन नहीं कर सकते हैं लेकिन में युवा एथ्लीटो के संपून सबस्ते को महतुपुन रूप से प्रभावित कर सकते हैं ।

Reference: हम स